In [1]:
# 载入数据

import pandas as pd
import copy

from utils.consts import *

train_data = pd.read_csv(TRAIN_DATA_PATH).drop(columns="tconst")
test_data = pd.read_csv(TEST_DATA_PATH).drop(columns="tconst")


> 基于相似度的评分模型

In [2]:
train_data_simi = copy.deepcopy(train_data)
test_data_simi = copy.deepcopy(test_data)

# 均值填充缺失值
def fillna(df):
    for column in list(df.columns[df.isnull().sum() > 0]):
        mean_val = df[column].mean()
        df[column].fillna(mean_val, inplace=True)

fillna(train_data_simi)
fillna(test_data_simi)

# 剔除训练集中评分人数少于50的电影
train_data_simi = train_data_simi.drop(train_data_simi[train_data_simi["numVotes"]<50].index)

# 剔除评分列
train_data_simi.drop(columns="averageRating")
test_data_simi.drop(columns="averageRating")

# 归一化
train_data_simi = (train_data_simi - train_data_simi.min())/(train_data_simi.max()-train_data_simi.min())
test_data_simi = (test_data_simi - test_data_simi.min())/(test_data_simi.max()-test_data_simi.min())

In [3]:
# 将数据转换为numpy矩阵
import numpy as np

train_matrix = np.matrix(train_data_simi)
test_matrix = np.matrix(test_data_simi)

In [9]:
def simi_predict(vec):
    """
    预测函数,计算给定向量vec和train_matrix之间的欧氏距离,取其中最相似的K个电影的评分均值作为预测评分
    """
    vec = np.array(vec)
    vec_2_matrix = np.repeat(vec,len(train_matrix),axis=0)
    sub_matrix = vec_2_matrix - train_matrix
    dis=[]
    for row in sub_matrix:
        row = np.array(row)
        dis.append(float(np.dot(row,row.T)))
    max_num = max(dis)
    min_index_arr=[]
    for i in range(K):
        min_num = min(dis)
        min_index = dis.index(min_num)
        min_index_arr.append(min_index)
        dis[min_index]=max_num
    score_sum = 0
    for index in min_index_arr:
        score_sum += train_data["averageRating"][index]
    return score_sum/K

In [12]:
# 计算MSE

from tqdm import tqdm
from sklearn.metrics import mean_squared_error

test_scores = [0]*len(test_matrix)
for i in tqdm(range(len(test_matrix))):
    test_scores[i]=simi_predict(test_matrix[i])

mse_simi = mean_squared_error(y_true=test_data["averageRating"],y_pred=test_scores)
print(f"mse_simi: {mse_simi}")

  0%|          | 34/34130 [00:52<14:36:23,  1.54s/it]


KeyboardInterrupt: 